In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.31.0 --progress-bar off
!pip install -qqq langchain==0.0.266 --progress-bar off
!pip install -qqq chromadb==0.4.5 --progress-bar off
!pip install -qqq pypdf==3.15.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq InstructorEmbedding==1.0.1 --progress-bar off
!pip install -qqq pdf2image==1.16.3 --progress-bar off

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's

In [2]:
!wget -q https://github.com/PanQiWei/AutoGPTQ/releases/download/v0.4.1/auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl

In [3]:
!pip install -qqq auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl --progress-bar off

In [4]:
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 31 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.3 [186 kB]
Fetched 186 kB in 8s (23.9 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 121730

In [5]:
!pip install poppler-utils

In [6]:
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path
from transformers import AutoTokenizer, TextStreamer, pipeline

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

## Data

In [7]:
!rm -rf "db"

In [8]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [9]:
pdf_file_path = '/content/drive/MyDrive/povs/'
loader = PyPDFDirectoryLoader(pdf_file_path)
docs = loader.load()
len(docs)

175

In [10]:
embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

449

In [12]:
%%time
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

CPU times: user 24.9 s, sys: 532 ms, total: 25.4 s
Wall time: 30.7 s


## Llama 2-13B

In [13]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device=DEVICE,
    quantize_config=None,
)

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.


config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

quantize_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

In [14]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [15]:
# cuda 11.8 version
!pip install -U xformers --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.1/218.1 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.7/811.7 MB 2.4 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [16]:
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

    PyTorch 2.0.1+cu118 with CUDA 1108 (you have 2.1.0+cu121)
    Python  3.10.11 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'Marian

In [17]:
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 10})

In [18]:

DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant.""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <>
{system_prompt}
<>

{prompt} [/INST]
""".strip()

SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [19]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

## Chat with Multiple PDFs

In [20]:
result = qa_chain("What is deloitte's insight about From bytes to barrels?")

 Based on the given context, I can see that "From bytes to barrels" is a report by the Deloitte Center for Energy Solutions regarding the digital transformation in the upstream oil and gas industry. Specifically, it suggests that there will be a significant increase in the use of digital technologies in the industry over the next two to three years. However, without more information or specific details from the report, I cannot provide any further insights into what exactly is meant by "From bytes to barrels."


In [21]:
result = qa_chain("who are the authors of 'From bytes to barrels' based on the info about the authors section?")

 Based on the information provided in the "Authors" section, the authors of "From Bytes to Barrels" are:

1. Patricia Danielecki
2. Sean Collins
3. Neerav Shah
4. Alice Hartnett
5. Seth Raskin
6. Paul Kaiser.


In [22]:


result = qa_chain("who to be contacted if there is any question about integrated approach to combat cyber risk   article?")

 Based on the information provided in the text, the authors of the article are:

* Rajeev Chopra (global leader - energy & resources at Deloitte Touche Tohmatsu Limited)
* Steve Livingston (national power & utilities - risk advisory leader at Deloitte US)
* Ramsey Hajj (risk advisory - senior manager at Deloitte US)
* Marko Van Zwam (risk advisory - partner at Deloitte Netherlands)
* Charles Hosner (risk advisory - partner at Deloitte UK)
* Rob Hayes (risk advisory - director at Deloitte UK)

Therefore, if you have any questions about the integrated approach to combat cyber risk in the oil and gas industry, you can contact any of these individuals.


In [23]:
len(result["source_documents"])

2

In [24]:
print(result["source_documents"][1].page_content)


+27 11 806 5167
tvanschalkwyk@deloitte.co.zaRajeev Chopra
Global Leader – Energy & Resources
Deloitte Touche Tohmatsu Limited
+44 20 7007 2933
rchopra@deloitte.co.uk
Steve Livingston
National Power & Utilities  
– Risk Advisory Leader
Deloitte US
+1 206 716 7539
slivingston@deloitte.com
Ramsey Hajj
Risk Advisory  
– Senior Manager
Deloitte US
+1 561 962 7843
rhajj@deloitte.com
Marko Van Zwam 
Risk Advisory – Partner
Deloitte Netherlands
+31 88 288 0890
MvanZwam@deloitte.nl
Charles Hosner 
Risk Advisory – Partner
Deloitte UK
+44 20 7007 2827
chosner@deloitte.co.uk
Rob Hayes 
Risk Advisory – Director
Deloitte UK
+44 20 7007 2606
rjhayes@deloitte.co.uk
12An integrated approach to combat cyber risk  | Securing industrial operations in oil and gas


In [25]:
result=qa_chain("what do you know about the authors of 2023 FSI Predictions")

 Based on the information provided, I can tell you the following about the authors of the 2023 FSI Predictions:

* The authors are from the Deloitte Center for Financial Services.
* They have made predictions for the financial services industry (FSI) in the year 2023.
* The predictions cover a range of topics, including quantum computing, embedded insurance, property conversions, carbon offset purchases, alternative data, and higher deposit costs.

I don't have any additional information about the authors beyond what is provided in the table of contents.


In [26]:
result=qa_chain("what are the names of the authors of 2023 FSI Predictions")

 I don't know the answer to your question. The text provided does not list the names of the authors of the "2023 FSI Predictions."


In [27]:
result=qa_chain("who is Monica O'Reilly based on the information provided in 2023 FSI Predictions")

 Based on the information provided in 2023 FSI Predictions, Monica O'Reilly is the US Financial Services Industry leader.


In [28]:
result=qa_chain("who is Krissy Davis?")

 Based on the information provided, Krissy Davis is the leader of Deloitte's US investment management industry practice and a Vice Chair of Deloitte.


In [29]:
result=qa_chain("What is Deloitte's predictions for 2023")

 Based on the information provided in the text, Deloitte's predictions for 2023 include:

1. The global investment management industry looking to drive success in a virtuous cycle.
2. Financial inclusion being a key theme, with a focus on looking beyond traditional labels.
3. A desire for financial advice among nearly 3 in 5 Americans, but uncertainty about where to obtain it.
4. The robo-advisory model not working out exactly as planned.
5. Banks and capital markets continuing to evolve and adapt to changing market conditions.

It is important to note that these predictions are based on Deloitte's analysis and may not be entirely accurate or comprehensive. Additionally, there may be other factors and trends that could impact the financial services industry in 2023 that are not mentioned in the text.


In [30]:
print(result["source_documents"][0].page_content)

1
2023 FSI Predictions
2023 FSI PredictionsEmerging developments and trends in the 
financial services industry (FSI)
Deloitte Center for Financial Services
